In [ ]:
from pyspark.sql.functions import col, when, lit, avg

# Read from silver_combined_indicators
df = spark.read.table("civAI.silver.silver_combined_indicators")

# DRPI Component Scores
df = df.withColumn("exposure_score", when(col("disaster_type").isNotNull(), 0.7).otherwise(0.3))
df = df.withColumn("infra_score", when(col("capacity") > 1000, 0.9).otherwise(0.5))
df = df.withColumn("policy_score", when(col("budget_allocation_musd") > 10, 0.8).otherwise(0.4))

# Use normalized vulnerability_score as-is
df = df.withColumn("vulnerability_score", col("vulnerability_score").cast("double"))

# Final DRPI Score
df = df.withColumn("drpi_score",
    (col("exposure_score") * 0.30) +
    (col("vulnerability_score") * 0.30) +
    (col("infra_score") * 0.20) +
    (col("policy_score") * 0.20)
)

# Select essential columns
df_result = df.select(
    "region", "disaster_type", "start_date", "drpi_score"
).distinct()

# Write to Gold Table
df_result.write.format("delta").mode("overwrite").saveAsTable("civAI.gold.drpi_scores")
